## Итоговые метрики сравнения сцен по объектам и атрибуами

Итоговая реализация в **final_metrics.py**

На вход подаются 2 сцены вида

```
{
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}
```

метрика выдает:
```
"f1_objects": 
"f1_attributes_macro": 
"f1_attributes_weighted": 
"f1_global_obj_attr_pairs": 
"f1_combined_simple": 
"f1_combined_weighted": 
```

In [39]:
import spacy
from typing import List, Dict, Tuple
from collections import Counter

# Загружаем модель spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

In [40]:
def lemmatize_scene(scene: Dict, nlp) -> Dict:
    def lemmatize(text: str) -> str:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

    new_scene = {
        "scene": {
            "location": scene["scene"].get("location", ""),
            "objects": [],
            "relations": []
        }
    }

    # Лемматизируем объекты и их признаки
    for obj in scene["scene"].get("objects", []):
        for obj_name, attributes in obj.items():
            obj_lemma = lemmatize(obj_name)
            attrs_lemma = [lemmatize(attr) for attr in attributes]
            new_scene["scene"]["objects"].append({obj_lemma: attrs_lemma})

    # Лемматизируем связи
    for subj, rel, obj in scene["scene"].get("relations", []):
        subj_lemma = lemmatize(subj)
        rel_lemma = lemmatize(rel)
        obj_lemma = lemmatize(obj)
        new_scene["scene"]["relations"].append([subj_lemma, rel_lemma, obj_lemma])

    return new_scene

In [62]:
def precision_recall_f1(tp: int, fp: int, fn: int) -> Tuple[float, float, float]:
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1

def evaluate_obj_attr_metrics(pred, label, normalize = True) -> Dict[str, float]:
    
    # лемматизируем если нужно
    if normalize:
        nlp = spacy.load("ru_core_news_sm")
        pred = lemmatize_scene(pred,nlp)
        label = lemmatize_scene(label,nlp)

    # F1 по объектам
    pred_objects = set([list(descr.keys())[0] for descr in  pred["scene"]["objects"]])
    label_objects = set([list(descr.keys())[0] for descr in  label["scene"]["objects"]])
    
    #print(pred_objects, label_objects)
    # до сих пор ок
    
    tp_obj = len(pred_objects & label_objects)
    fp_obj = len(pred_objects - label_objects)
    fn_obj = len(label_objects - pred_objects)
    _, _, f1_objects = precision_recall_f1(tp_obj, fp_obj, fn_obj)

    # F1 по признакам по каждому объекту (усреднение по объектам, macro)
    f1_per_object = []
    total_attrs = 0
    weighted_sum = 0
    
    for obj in label_objects | pred_objects:
        # по объекту из пересечения извлекаем атрибуты 
        # ну так себе конечно, можно и переписать но в целом ок, хотя и легаст
        # суть в том что в одном из объекта может не быть - тогда и атрибутов нет
        try:
            label_attrs  = set([obj_dict[obj] for obj_dict in label["scene"]["objects"] if obj in obj_dict.keys()][0])
        except:
            label_attrs = set()
        try:    
            pred_attrs  = set([obj_dict[obj] for obj_dict in pred["scene"]["objects"] if obj in obj_dict.keys()][0])
        except:
            pred_attrs = set()
            
        #print(pred_attrs, label_attrs)
        
        tp = len(label_attrs & pred_attrs)
        fp = len(pred_attrs - label_attrs)
        fn = len(label_attrs - pred_attrs)
        _, _, f1 = precision_recall_f1(tp, fp, fn)
        f1_per_object.append(f1)
        weighted_sum += f1 * len(label_attrs)
        total_attrs += len(label_attrs)

    f1_attributes_macro = sum(f1_per_object) / len(f1_per_object) if f1_per_object else 0.0
    f1_attributes_weighted = weighted_sum / total_attrs if total_attrs > 0 else 0.0

    # Глобальный F1 по парам (obj, attr)
    pred_pairs = {(obj, attr) for obj, attrs in pred.items() for attr in attrs}
    label_pairs = {(obj, attr) for obj, attrs in label.items() for attr in attrs}
    tp_pairs = len(pred_pairs & label_pairs)
    fp_pairs = len(pred_pairs - label_pairs)
    fn_pairs = len(label_pairs - pred_pairs)
    _, _, f1_global_pairs = precision_recall_f1(tp_pairs, fp_pairs, fn_pairs)

    # Объединённые метрики
    f1_combined_simple = (f1_objects + f1_attributes_macro) / 2
    total_obj = len(label_objects)
    f1_combined_weighted = ((total_obj * f1_objects) + (total_attrs * f1_attributes_weighted)) / (total_obj + total_attrs) if (total_obj + total_attrs) > 0 else 0.0

    return {
        "f1_objects": round(f1_objects, 4),
        "f1_attributes_macro": round(f1_attributes_macro, 4),
        "f1_attributes_weighted": round(f1_attributes_weighted, 4),
        "f1_global_obj_attr_pairs": round(f1_global_pairs, 4),
        "f1_combined_simple": round(f1_combined_simple, 4),
        "f1_combined_weighted": round(f1_combined_weighted, 4),
    }


### Проверка

In [63]:
scene_1 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

scene_2 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

evaluate_obj_attr_metrics(scene_1, scene_2)

{'f1_objects': 1.0,
 'f1_attributes_macro': 0.6667,
 'f1_attributes_weighted': 1.0,
 'f1_global_obj_attr_pairs': 1.0,
 'f1_combined_simple': 0.8333,
 'f1_combined_weighted': 1.0}

In [64]:
scene_1 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

scene_2 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"снеговик": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"бутылка": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

evaluate_obj_attr_metrics(scene_1, scene_2)

{'f1_objects': 0.3333,
 'f1_attributes_macro': 0.2,
 'f1_attributes_weighted': 0.75,
 'f1_global_obj_attr_pairs': 1.0,
 'f1_combined_simple': 0.2667,
 'f1_combined_weighted': 0.5714}